<a href="https://colab.research.google.com/github/kgabriella/FUNSD_OCR/blob/main/Projection_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
from google.colab import drive
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

import cv2
import numpy as np
import pandas as pd
import os
from PIL import Image
from google.colab.patches import cv2_imshow


In [ ]:
# mounting my drive to access images and annotations
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataframe = pd.read_excel('/content/drive/MyDrive/OCR_project/All_bounding_boxes.xlsx')
root_dir = '/content/drive/MyDrive/OCR_project/'

In [ ]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import pandas as pd

# Preprocess the image using Adaptive Thresholding
def preprocess_image(cropped_img_np):
    if cropped_img_np.ndim == 2:
        gray = cropped_img_np
    elif cropped_img_np.shape[2] == 4:
        cropped_img_np = cv2.cvtColor(cropped_img_np, cv2.COLOR_RGBA2RGB)
        gray = cv2.cvtColor(cropped_img_np, cv2.COLOR_RGB2GRAY)
    elif cropped_img_np.shape[2] == 3:
        gray = cv2.cvtColor(cropped_img_np, cv2.COLOR_RGB2GRAY)
    else:
        raise ValueError("Unexpected number of channels in the input image")

    # Apply Adaptive Gaussian Thresholding
    binary_inv = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY_INV, 11, 2)

    return gray, binary_inv

# Segment lines based on horizontal projection
def segment_lines(binary_image):
    horizontal_projection = np.sum(binary_image, axis=1)
    lines = []
    in_line = False
    line_start = 0
    threshold = binary_image.shape[1] * 0.1  # Minimum pixels to consider it as a line

    for i, value in enumerate(horizontal_projection):
        if value > threshold and not in_line:
            line_start = i
            in_line = True
        elif value <= threshold and in_line:
            line_end = i
            if line_end - line_start >= 3:  # Exclude lines with height less than 3 pixels
                lines.append((line_start, line_end))
            in_line = False

    if in_line:
        if i - line_start >= 3:  # Exclude lines with height less than 3 pixels
            lines.append((line_start, i))

    return lines

# Segment characters based on vertical projection within a line
def segment_characters(line_image):
    vertical_projection = np.sum(line_image, axis=0)
    characters = []
    in_character = False
    character_start = 0

    for i, value in enumerate(vertical_projection):
        if value > 0 and not in_character:
            character_start = i
            in_character = True
        elif value == 0 and in_character:
            character_end = i
            characters.append((character_start, character_end))
            in_character = False

    if in_character:
        characters.append((character_start, i))

    return characters, vertical_projection

# Refine character segmentation if the detected count is less than expected
def refine_segmentation(line_image, characters, vertical_projection, expected_count, multi_line):
    if len(characters) >= expected_count:
        return characters

    split_needed = expected_count - len(characters)

    if multi_line:
        split_needed = min(split_needed, 3)  # Limit to 3 splits for multiple lines

    while split_needed > 0:
        # Find the widest character box
        widest_boxes = sorted(characters, key=lambda x: x[1] - x[0], reverse=True)
        start, end = widest_boxes[0]
        width = end - start

        # Only split one box if one character is missing
        if multi_line and split_needed == 1:
            split_count = 2
        else:
            split_count = min(split_needed + 1, 2)  # Add at most two splits at a time

        step = width // split_count

        new_splits = [(start + i * step, start + (i + 1) * step) for i in range(split_count)]
        if new_splits[-1][1] < end:
            new_splits[-1] = (new_splits[-1][0], end)

        characters = characters[:characters.index(widest_boxes[0])] + new_splits + characters[characters.index(widest_boxes[0]) + 1:]
        split_needed -= (split_count - 1)

        if multi_line and split_needed <= 0:
            break

    return characters

# Function to save character images to corresponding folders
def save_characters_to_folders(char_images, label, image_path):
    if not isinstance(label, str):
        label = str(label)

    map_dict = {}
    for char_image, char in zip(char_images, label.replace(" ", "")):
        if char_image.size == 0:
            continue
        folder_path = f'/content/drive/MyDrive/OCR_project/segmented/{char}'
        os.makedirs(folder_path, exist_ok=True)
        if char in map_dict:
            map_dict[char] += 1
        else:
            map_dict[char] = 0
        file_path = os.path.join(folder_path, f"{char}_v2_{map_dict[char]}_{os.path.basename(image_path)}.png")
        cv2.imwrite(file_path, char_image)

# Function to process and save segmented characters
def process_and_save_characters(img_path, coordinates, label):
    image = Image.open(img_path)
    left, top, right, bottom = eval(coordinates)
    expected_char_count = len(str(label).replace(" ", ""))  # Ignore spaces in the label

    img = image.crop((left, top, right, bottom))
    width = right - left
    height = bottom - top
    if height > width * 1.5:
        img = img.rotate(90, expand=True)  # Rotate the image if it's taller than it is wide
    cropped_img_np = np.array(img)

    gray, binary_inv = preprocess_image(cropped_img_np)
    lines = segment_lines(binary_inv)
    multi_line = len(lines) > 1  # Check if there are multiple lines

    characters_all_lines = []
    final_char_images = []
    for line in lines:
        line_image = binary_inv[line[0]:line[1], :]
        characters, vertical_projection = segment_characters(line_image)
        characters_all_lines.append((line, characters, vertical_projection))

        detected_char_count = len(characters)
        if detected_char_count != expected_char_count and detected_char_count <= expected_char_count + 6:
            refined_characters = refine_segmentation(line_image, characters, vertical_projection, expected_char_count, multi_line)
        else:
            refined_characters = characters

        for char_start, char_end in refined_characters:
            x1 = max(char_start, 0)
            y1 = max(line[0], 0)
            x2 = min(char_end, binary_inv.shape[1])
            y2 = min(line[1], binary_inv.shape[0])
            final_char_images.append(cropped_img_np[y1:y2, x1:x2])

    if len(final_char_images) == expected_char_count:
        save_characters_to_folders(final_char_images, label, img_path)

def show_image_with_box(df, idx, root_dir):
    img_path = os.path.join(root_dir, df.iloc[idx]['file path'])
    coordinates = df.iloc[idx]['coordinates']
    label = df.iloc[idx]['label']
    process_and_save_characters(img_path, coordinates, label)

    image = Image.open(img_path)
    left, top, right, bottom = eval(coordinates)
    img = image.crop((left, top, right, bottom))
    cropped_img_np = np.array(img)
    width = right - left
    height = bottom - top
    if height > width * 1.5:
        cropped_img_np = cv2.rotate(cropped_img_np, cv2.ROTATE_90_COUNTERCLOCKWISE)
    gray, binary_inv = preprocess_image(cropped_img_np)
    lines = segment_lines(binary_inv)

    fig, axes = plt.subplots(1, 4, figsize=(24, 6))
    axes[0].imshow(gray, cmap='gray')
    axes[0].set_title("Grayscale Image")

    axes[1].imshow(binary_inv, cmap='gray')
    axes[1].set_title("Binary Image (Adaptive Thresholding)")

    ax_img_chars = axes[2]
    ax_img_chars.imshow(binary_inv, cmap='gray')
    ax_img_chars.set_title("Detected Characters")

    ax_img_refined = axes[3]
    ax_img_refined.imshow(binary_inv, cmap='gray')
    ax_img_refined.set_title("Refined Segmentation")

    padding = 1
    characters_all_lines = []
    for line in lines:
        line_image = binary_inv[line[0]:line[1], :]
        characters, vertical_projection = segment_characters(line_image)
        characters_all_lines.append((line, characters, vertical_projection))

        for char_start, char_end in characters:
            x1 = max(char_start - padding, 0)
            y1 = max(line[0] - padding, 0)
            x2 = min(char_end, binary_inv.shape[1])
            y2 = min(line[1] + padding, binary_inv.shape[0])
            rect_char = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, edgecolor='red', facecolor='none')
            ax_img_chars.add_patch(rect_char)

    for line, characters, vertical_projection in characters_all_lines:
        detected_char_count = len(characters)
        expected_char_count = len(str(label).replace(" ", ""))
        multi_line = len(lines) > 1
        if detected_char_count != expected_char_count and detected_char_count <= expected_char_count + 6:
            refined_characters = refine_segmentation(line_image, characters, vertical_projection, expected_char_count, multi_line)

            for char_start, char_end in refined_characters:
                x1 = max(char_start - padding, 0)
                y1 = max(line[0] - padding, 0)
                x2 = min(char_end, binary_inv.shape[1])
                y2 = min(line[1] + padding, binary_inv.shape[0])
                rect_refined_char = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, edgecolor='green', facecolor='none')
                ax_img_refined.add_patch(rect_refined_char)

    plt.tight_layout()
    plt.show()

# Load the Excel file
file_path = '/content/drive/MyDrive/OCR_project/All_bounding_boxes.xlsx'
data = pd.read_excel(file_path)

# Iterate through the DataFrame and visualize each image with bounding boxes
for i in range(len(data)):
    show_image_with_box(data, i, '/content/drive/MyDrive/OCR_project/FUNSD_dataset/training_data/images/')
